Step 1: Load all relevant libraries

In [2]:
from glob import glob
import requests
import shutil
import os
import wget 

import earthpy as et
import earthpy.plot as ep
import earthpy.appeears as eaap
import folium
import geopandas as gpd
import geoviews as gv
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.plot import show
import rioxarray as rxr
import rioxarray.merge as rxm
import xarray as xr
from xrspatial import aspect
from xrspatial import slope

data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
grasslands_dir = os.path.join(data_dir, 'grassland_units')
vars_dir = os.path.join(data_dir, 'variables')
soil_dir = os.path.join(vars_dir, 'theta_r_layer')

for a_dir in [grasslands_dir, vars_dir, soil_dir]:
    if not os.path.exists(a_dir):
        os.makedirs(a_dir)

Step 2: Load the National Grassland Units

In [3]:
# Downloading and saving the grasslands shapefile to the grasslands directory
grasslands_path = os.path.join(grasslands_dir, 'S_USA.NationalGrassland.shp')
if not os.path.exists(grasslands_path):
    gpd.read_file("https://data.fs.usda.gov/geodata/edw/edw_resources/shp/S_USA.NationalGrassland.zip").to_file(grasslands_path)

# Building a GeoDataFrame from the USFS grasslands shapefile
grasslands_gdf = (gpd
                  .read_file(grasslands_path)
                  .set_index('GRASSLANDN'))

# Subsetting the Little Missouri and Crooked River NG's
lm_cr_s_ng_gdf = (
    grasslands_gdf
    .loc[['Little Missouri National Grassland', 'Crooked River National Grassland', 'Sheyenne National Grassland']]
)
lmng_utm = 32614
lmng_gdf = lm_cr_s_ng_gdf.loc[['Little Missouri National Grassland']]
crng_utm = 32613
crng_gdf = lm_cr_s_ng_gdf.loc[['Crooked River National Grassland']]
sng_utm = 32611
sng_gdf = lm_cr_s_ng_gdf.loc[['Sheyenne National Grassland']]


Step 3: Downloading a series of variable layers in order to build the suitability model

In [4]:
# Load the soil residual water content (theta_r) variable raster layer from the Polaris Dataset
def polaris_theta_r_download(min_lat, max_lat, min_lon, max_lon, file_name):
    """
    Parameters
    ==========
    min_lat : integer
        The minimum single latitude which bounds the desired sudy area
    min_lon : integer
        The minimum single longitue which bounds the desired study area

    Returns
    =======
    DataArray of raster data for the desired study variable bounded by a single degree of latitue and longitude
    """
    soil_fctn_path = os.path.join(soil_dir, file_name)
    if not os.path.exists(soil_fctn_path):
        lat_range = list(range(min_lat, max_lat + 1))
        print(lat_range)
        lon_range = list(range(max_lon, min_lon + 1))
        print(lon_range)
        polaris_das = []
        for lat in lat_range:
            for lon in lon_range:
                polaris_url_format = ("http://hydrology.cee.duke.edu/POLARIS/PROPERTIES"
                                      "/v1.0/theta_r/mean/100_200/lat{min_lat}{max_lat}"
                                      "_lon{min_lon}{max_lon}.tif")

                polaris_url = polaris_url_format.format(
                min_lat=lat, min_lon=lon-1, max_lat=lat+1, max_lon=lon
                )

                print(polaris_url)

                polaris_das.append(rxr.open_rasterio(polaris_url, masked=True).squeeze())
        rxm.merge_arrays(polaris_das).rio.to_raster(soil_fctn_path, driver='GTiff')

    return rxr.open_rasterio(soil_fctn_path)


In [5]:
def raster_clip_reproject(da, utm, gdf):
    """
    Parameters
    ==========
    da : DataArray
        Raster array to be clipped and reprojected into the UTM CRS
    utm : integer
        The correct epsg for the utm reprojection
    gdf : DeoDataFrame
        The geodatafrma whose geometry and bounds will be used to clip the raster data

    Returns
    =======
    The raster data array that has been clipped and reprojected
    """
    gdf = gdf.to_crs(utm)
    da_adj = (da
              .rio.clip_box(*gdf.total_bounds)
              .rio.reproject(utm)
    )
    return da_adj

In [6]:
# lmng_soil_da = polaris_theta_r_download(46, 48, -102, -104, 'lmng-soil.tif')

crng_soil_da = polaris_theta_r_download(44, 45, -121, -122, 'crng-soil.tif')

sng_soil_da = polaris_theta_r_download(46, 47, -97, -98, 'sng-soil.tif')


In [7]:
crng_soil_da.hvplot()

: 

In [ ]:
lmng_soil_crp_da = raster_clip_reproject(lmng_soil_da, lmng_utm, lmng_gdf)
crng_soil_crp_da = raster_clip_reproject(crng_soil_da, crng_utm, crng_gdf)
sng_soil_crp_da = raster_clip_reproject(sng_soil_da, sng_utm, sng_gdf)

In [ ]:
# Creating a for loop to replace the above function
# List of national grasslands with associated latitude and longitude
# national_grasslands_minlatlon = [
#     {"name": "Little Missouri National Grassland", "latitude": 47, "longitude": -103},
#     {"name": "Crooked River National Grassland", "latitude": 44, "longitude": -121},
#     {"name": "Sheyenne National Grassland", "latitude": 46, "longitude": -97},
# ]

# # Loop through each national grassland
# for grassland in national_grasslands_minlatlon:
#     name = grassland["name"]
#     min_lat, min_lon = grassland["latitude"], grassland["longitude"]
#     max_lat, max_lon = min_lat+1, min_lon+1

#     polaris_url_format = ("http://hydrology.cee.duke.edu/POLARIS/PROPERTIES"
#                           "/v1.0/theta_r/mean/100_200/lat{min_lat}{max_lat}"
#                           "_lon{min_lon}{max_lon}.tif")

#     polaris_url = polaris_url_format.format(
#     min_lat=min_lat, min_lon=min_lon, max_lat=max_lat, max_lon=max_lon
#     )

#     print(polaris_url)

#     data_array = rxr.open_rasterio(polaris_url, masked=True).squeeze()

#     # Print information about the data array
#     print(f"DataArray for {name}:")
#     print(data_array)

#     # Optionally, save the DataArray as a variable
#     locals()[f"{name.lower()}_data"] = data_array



In [ ]:
# Load elevation data from the SRTM dataset through the APPEEARS API

def srtm_download(file_name, gdf):
    """
    Parameters
    ==========
    file_name : str
        Desired file name for the DataArray when it is saved in the earthpy downloads directory
    gdf : GeoDatafram
        The dataframe conatining a single geometry for the APPEEARS downloader to indentify 

    Returns
    =======
    DataArray of the elevations raster data for the selected national grassland geometry
    """
    srtm_downloader = eaap.AppeearsDownloader(
    ea_dir=data_dir,
    download_key=file_name,
    product='SRTMGL1_NC.003',
    layer='SRTMGL1_DEM',
    start_date='02-11-2000',
    end_date='02-21-2000',
    polygon=gdf
    )
    if not os.path.exists(os.path.join(data_dir, file_name)):
        srtm_downloader.download_files(cache=True)

    srtm_filepaths = glob(os.path.join(srtm_downloader.data_dir,
                                       'SRTMGL1_NC.003*',
                                       '*.tif'
                                       ))
    return rxm.merge_arrays([rxr.open_rasterio(srtm_filepaths, masked=True).squeeze() for srtm_filepaths in srtm_filepaths])

lmng_elev_da = srtm_download('little-missouri-ng-srtm', lmng_gdf)
crng_elev_da = srtm_download('crooked-river-ng-srtm', crng_gdf)
sng_elev_da = srtm_download('sheyenne-ng-srtm', sng_gdf)

In [ ]:
lmng_elev_crp_da = raster_clip_reproject(lmng_elev_da, lmng_utm, lmng_gdf)
crng_elev_crp_da = raster_clip_reproject(crng_elev_da, crng_utm, crng_gdf)
sng_elev_crp_da = raster_clip_reproject(sng_elev_da, sng_utm, sng_gdf)

In [ ]:
sng_elev_da.rio.clip_box(*sng_gdf.total_bounds).rio.reproject_match(sng_soil_crp_da)

sng_elev_da.hvplot()

In [ ]:
# Load and save climate variable raster layers from the MACAv2 Dataset

maca_url = ("http://thredds.northwestknowledge.net:8080/thredds/ncss/"
              "agg_macav2metdata_pr_CCSM4_r6i1p1_rcp45_2006_2099_CONUS_monthly"
              ".nc?var=precipitation&disableLLSubset=on"
              "&disableProjSubset=on&horizStride=1"
              "&time_start=2010-01-15T00%3A00%3A00Z"
              "&time_end=2010-02-15T00%3A00%3A00Z"
              "&timeStride=1&accept=netcdf"
)

maca_respoinse = requests.get(maca_url)
with open('maca.nc', 'wb') as maca_file:
    maca_file.write(maca_respoinse.content)

In [ ]:
maca_ds = xr.open_dataset('maca.nc')
maca_ds = maca_ds.assign_coords(lon=maca_ds.lon-360)
precip_da = maca_ds.precipitation
precip_da = precip_da.rio.write_crs("epsg:4326", inplace=True)
precip_da = precip_da.rio.set_spatial_dims('lon', 'lat', inplace=True)

# precip_da.mean('time').hvplot(rasterize=True) * lmng_gdf.hvplot()

In [ ]:
# precip_da.rio.clip_box(*lmng_gdf.total_bounds).mean('time').hvplot()

lmng_precip_crp_da = raster_clip_reproject(precip_da, lmng_utm, lmng_gdf)
crng_precip_crp_da = raster_clip_reproject(precip_da, crng_utm, crng_gdf)
sng_precip_crp_da = raster_clip_reproject(precip_da, sng_utm, sng_gdf)

Step 4: calculating slope and aspect

In [ ]:
# utm_zone = 14, 13, 11

def elev_to_slope (da, utm_zone):
    reprojected_da = da.rio.reproject(f"EPSG:326{utm_zone}")
    print(reprojected_da)
    return slope(reprojected_da)

sng_slope_da = elev_to_slope(sng_elev_da, 14)

sng_gdf

In [ ]:
from xrspatial import aspect

def elev_to_aspect (da, utm_zone):
    reprojected_da = da.rio.reproject(f"EPSG:326{utm_zone}")
    return aspect(reprojected_da)

sng_aspect_da = elev_to_aspect(sng_elev_da, 14)

In [ ]:
# Reproject the second raster to match the first raster
# precip_da_utm = precip_da.rio.reproject_match(sng_slope_da)
# sng_soil_da_utm = sng_soil_da.rio.reproject_match(sng_slope_da)

